# Analyzing bot data with Jupyter notebooks  

You can analyze the results of backtests and trading history easily using Jupyter notebooks. A sample notebook is located at `user_data/notebooks/analysis_example.ipynb`.  

## Pro tips  

* See [jupyter.org](https://jupyter.org/documentation) for usage instructions.
* Don't forget to start a jupyter notbook server from within your conda or venv environment or use [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels)*
* Copy the example notebook so your changes don't get clobbered with the next freqtrade update.

## Fine print  

Some tasks don't work especially well in notebooks. For example, anything using asyncronous exectution is a problem for Jupyter. Also, freqtrade's primary entry point is the shell cli, so using pure python in a notebook bypasses arguments that provide required parameters to functions.

## Recommended workflow  

| Task | Tool |  
  --- | ---  
Bot operations | CLI  
Repetative tasks | shell scripts
Data analysis & visualization | Notebook  

1. Use the CLI to
    * download historical data
    * run a backtest
    * run with real-time data
    * export results  

1. Collect these actions in shell scripts
    * save complicated commands with arguments
    * execute mult-step operations  
    * automate testing strategies and prepareing data for analysis

1. Use a notebook to
    * import data
    * munge and plot to generate insights

## Example utility snippets for Jupyter notebooks

### Change directory to root  

Jupyter notebooks execute from the notebook directory. The following snippet searches for the project root, so relative paths remain consistant.

In [ ]:
# Change directory
# Modify this cell to insure that the output shows the correct path.
# Define all paths relative to the project root shown in the cell output
import os
from pathlib import Path

project_root = "somedir/freqtrade"
i=0
try:
    os.chdirdir(project_root)
    assert Path('LICENSE').is_file()
except:
    while i<4 and (not Path('LICENSE').is_file()):
        os.chdir(Path(Path.cwd(), '../'))
        i+=1
    project_root = Path.cwd()
print(Path.cwd())

### Watch project for changes to code
This scans the project for changes to code before Jupyter runs cells.

In [ ]:
# Reloads local code changes
%load_ext autoreload
%autoreload 2

## Load existing objects into a Jupyter notebook

These examples assume that you have already generated data using the cli. These examples will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results into a pandas dataframe

In [ ]:
# Load backtest results
from freqtrade.data.btanalysis import load_backtest_data
df = load_backtest_data("user_data/backtest_data/backtest-result.json")

# Show value-counts per pair
df.groupby("pair")["sell_reason"].value_counts()

### Load live trading results into a pandas dataframe

In [ ]:
# Fetch trades from database
from freqtrade.data.btanalysis import load_trades_from_db
df = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
df.groupby("pair")["sell_reason"].value_counts()

### Load multiple configuration files
This option can be useful to inspect the results of passing in multiple configs

In [ ]:
# Load config from multiple files
from freqtrade.configuration import Configuration
config = Configuration.from_files(["config1.json", "config2.json"])

# Show the config in memory
import json
print(json.dumps(config, indent=1))

### Load exchange data to a pandas dataframe

This loads candle data to a dataframe

In [ ]:
# Load data using values passed to function
from pathlib import Path
from freqtrade.data.history import load_pair_history

ticker_interval = "5m"
data_location = Path('user_data', 'data', 'bitrex')
pair = "BTC_USDT"
candles = load_pair_history(datadir=data_location,
                            ticker_interval=ticker_interval,
                            pair=pair)

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")
display(candles.head())

Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.